In [1]:
import numpy as np
import pandas as pd
import openpyxl
import os 
import xlrd
import datetime
# Setting Directory
os.chdir('C:/Users/TAM/Documents/Vu Quang Nguyen/VACP/')

In [2]:
#BAN HANG
URL_ban="../VACP/Export BRAVO/Sổ chi tiết bán hàng.xls"  
data_banhang=pd.read_excel(URL_ban,skipfooter=1,header=6).rename(
    columns={
        "Unnamed: 0":"Ngày bán",
        "Unnamed: 1":"ID Order",
        "Unnamed: 2":"ID Khách hàng",
        "Unnamed: 3":"Hãng SX",
        "Unnamed: 5":"ID",
        "Unnamed: 7":"Tên hàng",
        "Tiền bán":"Thành tiền",
        "Unnamed: 19":"Nhân viên",
        "Unnamed: 22":"ID Nhóm hàng"

    }
)
data_banhang=data_banhang[["Ngày bán","ID Order","ID Khách hàng","ID Nhóm hàng","Nhân viên","ID","Tên hàng","Số lượng","Giá bán","Thành tiền"]]


#CONG NO
data_congno=pd.read_excel("../VACP/Export BRAVO/Báo cáo công nợ phải thu theo hạn thanh toán.xls",skipfooter=1,header=6).rename(
    columns={
        "Mã":"ID Khách hàng",
        "Số":"ID Order",
        "Unnamed: 15":"Tiền thu được"
    }
).dropna(subset=["Ngày","ID Order"]).reset_index(drop=True)


# Data NHẬP MUA BÁN
data_nhap_mua_ban=pd.read_excel("../VACP/Input & Export Campaign/Report_Nhap_Mua_Ban.xlsx",sheet_name="Nhập Mua Bán",usecols="B,F:G,S:V")

# DIM_Customer
dim_customer=pd.read_excel("../VACP/Export BRAVO/Data_KH_VAPC.xlsx",sheet_name='Data_KH')


In [3]:
#Tổng hợp đơn hàng và tính Margin, Tiền thu được
data_banhang_full=data_banhang.merge(data_nhap_mua_ban[["ID","Giá vốn"]],on="ID",how='left')
# Tính Profit
data_banhang_full["Profit"]=(data_banhang_full["Thành tiền"]-data_banhang_full["Giá vốn"]*data_banhang_full["Số lượng"]) 
# Tính margin 
data_banhang_full["Margin"]=data_banhang_full['Profit']/data_banhang_full["Thành tiền"]
# Tổng hợp
data_banhang_full=data_banhang_full.groupby(["Ngày bán","ID Order","ID Khách hàng"],as_index=False).agg({
            'Thành tiền':'sum',
            'Profit':'sum',
            'Margin':'mean'})
# Import Công nợ

data_banhang_full=data_banhang_full.merge(data_congno[["ID Order","ID Khách hàng","Tiền thu được"]],on=['ID Order','ID Khách hàng'],how='left').sort_values(["Ngày bán","Tiền thu được"],ascending=False).reset_index(drop=True)

data_banhang_full['Tiền thu được'] = data_banhang_full['Tiền thu được'].replace(np.nan, 0)

# Khả năng thanh toán = Tiền thu được / Công nợ
data_banhang_full["Cash ratio"]=data_banhang_full["Tiền thu được"]/(data_banhang_full["Thành tiền"])
#SHOW

data_banhang_full.head()



,Ngày bán,ID Order,ID Khách hàng,Thành tiền,Profit,Margin,Tiền thu được,Cash ratio
0,2022-11-23,VAPC231122-001,CHPT-MINHANH,14400000,1.603693e+06,0.111368,0.0,0.0
1,2022-11-23,VAPC231122-002,KL-HIEUPHAT-TM,1850000,1.856066e+05,0.100328,0.0,0.0
2,2022-11-23,VAPC231122-003,NN-CHINGUYET,1920000,5.534895e+05,0.288976,0.0,0.0
3,2022-11-23,VAPC231122-004,QUOCHUNG-KT,960000,5.950852e+05,0.619880,0.0,0.0
4,2022-11-23,VAPC231122-005,KHACHLE-HOANVU,1846000,6.339087e+05,0.356338,0.0,0.0


In [4]:
data_banhang_full.loc[(data_banhang_full["ID Khách hàng"]=='CHUBASG') & (data_banhang_full["Ngày bán"].dt.strftime("%Y-%m-%d")=='2022-06-17')]

,Ngày bán,ID Order,ID Khách hàng,Thành tiền,Profit,Margin,Tiền thu được,Cash ratio
11238,2022-06-17,VAPC170622-048,CHUBASG,98400000,-7.945821e+06,-0.085859,98400000.0,1.0


In [5]:
# Tổng hợp Doanh số theo ngày
data_sale=data_banhang_full.groupby(["Ngày bán","ID Khách hàng"],as_index=False).agg({
            'Thành tiền':'sum',
            'Tiền thu được':'sum',
            'Profit':'sum',
            'Margin':'mean',
            'Cash ratio':'mean',
            "ID Order": lambda x: x.nunique()}).rename(columns={"ID Order":"Tổng đơn hàng","Thành tiền":"Doanh số"}).reset_index(drop=True)

# data_sale=data_sale.merge(dim_customer[["ID Khách hàng","Phân hạng","Nhân viên phụ trách","Tên khách hàng","Tỉnh","Vùng miền"]],on="ID Khách hàng",how="left")
data_sale.head()

,Ngày bán,ID Khách hàng,Doanh số,Tiền thu được,Profit,Margin,Cash ratio,Tổng đơn hàng
0,2021-01-02,AEA,3150000,0.0,709064.514342,0.227208,0.0,1
1,2021-01-02,ANHCUONG,4620000,0.0,452231.257692,0.078745,0.0,1
2,2021-01-02,ASCSG,2100000,0.0,398047.542459,0.157718,0.0,3
3,2021-01-02,GARAGETHANHTAI,540000,0.0,66202.915806,0.188611,0.0,2
4,2021-01-02,KL-CONGDANH,3020000,0.0,174550.972018,0.057794,0.0,1


In [6]:
# Lọc mẫu dữ liệu
data_sale.loc[(data_sale['ID Khách hàng']=='ANHCUONG') & (data_sale["Ngày bán"].dt.strftime('%Y-%m-%d') == '2022-11-19')]

,Ngày bán,ID Khách hàng,Doanh số,Tiền thu được,Profit,Margin,Cash ratio,Tổng đơn hàng
20363,2022-11-19,ANHCUONG,5450000,1450000.0,1.324144e+06,0.268842,0.214815,3


In [7]:
data_sale.loc[(data_sale["ID Khách hàng"]=='TRITIN') & (data_sale["Ngày bán"].dt.strftime("%Y-%m-%d")=='2022-07-01')]

,Ngày bán,ID Khách hàng,Doanh số,Tiền thu được,Profit,Margin,Cash ratio,Tổng đơn hàng
14696,2022-07-01,TRITIN,1396000,1396000.0,-237656.227166,-182.448393,1.0,3


In [8]:
# Data Sale năm 2022

data_sale_total=data_sale.loc[data_sale["Ngày bán"].dt.strftime("%Y")=='2022'].groupby("ID Khách hàng",as_index=False).agg({
    'Doanh số':'sum',
    'Tiền thu được':'sum',
    'Profit':'sum',
    'Tổng đơn hàng':'sum',
    'Cash ratio':'mean'      
}).sort_values(["Doanh số","Profit"],ascending=False).reset_index(drop=True)

data_sale_total["Gross Profit Margin"]=data_sale_total["Profit"]/data_sale_total["Doanh số"]


data_sale_total

,ID Khách hàng,Doanh số,Tiền thu được,Profit,Tổng đơn hàng,Cash ratio,Gross Profit Margin
0,KL-QUANGSAIGON,4242390000,4.142390e+09,3.217892e+08,539,0.999078,0.075851
1,KIET-CHUBA,3359809400,2.895634e+09,3.656982e+08,863,0.841903,0.108845
2,HAIDAO,2956562060,2.837062e+09,9.490589e+07,48,0.948718,0.032100
3,CHPT-ANNHIEN,2919359000,2.508460e+09,4.348366e+08,1139,0.819702,0.148949
4,ASCSG,2336944000,2.169374e+09,3.613123e+08,1494,0.925128,0.154609
...,...,...,...,...,...,...,...
295,HOANVU,622000,6.220000e+05,1.594027e+05,1,1.000000,0.256274
296,KL-THANHHIEP,440000,4.400000e+05,1.759520e+04,1,1.000000,0.039989
297,KL-TRANLINH,224000,2.240000e+05,5.503808e+04,1,1.000000,0.245706
298,KL-MANHCUONG,200000,2.000000e+05,8.059084e+04,1,1.000000,0.402954


In [9]:
data_sale_filter=data_sale_total.query('`Gross Profit Margin`>=0.02 & `Tổng đơn hàng`>=10 & `Cash ratio`>=0.7').sort_values(["Doanh số","Gross Profit Margin"],ascending=False).reset_index(drop=True)
data_sale_filter=data_sale_filter.merge(dim_customer[["ID Khách hàng","Phân hạng","Nhân viên phụ trách","Tên khách hàng","Tỉnh","Vùng miền"]],on="ID Khách hàng",how="left")
data_sale_filter=data_sale_filter.loc[0:149,:]
data_sale_filter

,ID Khách hàng,Doanh số,Tiền thu được,Profit,Tổng đơn hàng,Cash ratio,Gross Profit Margin,Phân hạng,Nhân viên phụ trách,Tên khách hàng,Tỉnh,Vùng miền
0,KL-QUANGSAIGON,4242390000,4.142390e+09,3.217892e+08,539,0.999078,0.075851,CHPT-SL,HOANTRAN,CHPT Anh Quang Sài Gòn,Hồ Chí Minh,Đông Nam Bộ
1,KIET-CHUBA,3359809400,2.895634e+09,3.656982e+08,863,0.841903,0.108845,CHPT-SL,HOANTRAN,CHPT Tiêu Kiệt - Con chú Bá,Hồ Chí Minh,Đông Nam Bộ
2,HAIDAO,2956562060,2.837062e+09,9.490589e+07,48,0.948718,0.032100,CHPT-SL,HOANTRAN,CHPT HẢI ĐÀO,Hà Nội,Đồng bằng sông Hồng
3,CHPT-ANNHIEN,2919359000,2.508460e+09,4.348366e+08,1139,0.819702,0.148949,CHPT-LE,HOANTRAN,CHPT An Nhiên (Anh Hiếu),Hồ Chí Minh,Đông Nam Bộ
4,ASCSG,2336944000,2.169374e+09,3.613123e+08,1494,0.925128,0.154609,CHPT-SL,HOANVU,CHPT ASC Sài Gòn,Hồ Chí Minh,Đông Nam Bộ
...,...,...,...,...,...,...,...,...,...,...,...,...
145,KL-TIENDU-NT,42480000,4.248000e+07,6.233669e+06,15,1.000000,0.146744,CHPT-LE,HOANVU,Công ty TNHH đầu tư Tiên Du,Khánh hòa,Nam Trung Bộ
146,KL-HOANGGIASG,42300000,4.230000e+07,4.588547e+06,17,1.000000,0.108476,CHPT-LE,HOANVU,Công ty Hoàng Gia,Hồ Chí Minh,Đông Nam Bộ
147,KL-MINHCHAU,41510000,4.151000e+07,6.940609e+06,23,1.000000,0.167203,CHPT-LE,VINHNGUYEN,CHPT Minh Châu,Hồ Chí Minh,Đông Nam Bộ
148,KL-VIETLONG-CTY,41145000,4.114500e+07,6.521097e+06,21,1.000000,0.158491,CHPT-LE,HOANVU,Công ty TNHH TMDV Việt Long (anh Trung) - TM,Hồ Chí Minh,Đông Nam Bộ


In [ ]:
# EXPORT FILE DATA
with pd.ExcelWriter('../VACP/Input & Export Campaign/Bao_Cao_Doanh_So_Khach_Hang.xlsx') as writer:
    data_sale.to_excel(writer,sheet_name="Sales Daily")
    data_banhang_full.to_excel(writer,sheet_name="Data")
    data_sale_filter.to_excel(writer,sheet_name="Top 150")
    # data_report.to_excel(writer,sheet_name="Tổng hợp")
    # data_danhmuchanghoa.to_excel(writer,sheet_name="Danh mục hàng")
    # count_danh_muc.to_excel(writer,sheet_name="đến số lượng")
